### Serialbox Tutorial

In this notebook, we'll cover the basics to use Serialbox to extract data within a Fortran program that we'll use later to create a GT4Py port.

### Notebook Requirements

- Python v3.11.x to v3.12.x
- [NOAA/NASA Domain Specific Language Middleware](https://github.com/NOAA-GFDL/NDSL)
- `ipykernel==6.1.0`
- [`ipython_genutils`](https://pypi.org/project/ipython_genutils/)
- Fortran compiler that built Serialbox in `NDSL` middleware.

### Create Fortran code example

The following simply creates a `Fortran` directory where we'll store the Fortran code used to demonstrate Serialbox commands.

In [ ]:
%%bash

cd /home/ckung/Documents/Code/NDSL/examples/serialbox

if [ ! -d "./Fortran" ]; then
    mkdir Fortran
fi


Next we'll create the file `fillq2zero.F90` within the previously created `Fortran` directory.

In [ ]:
%%writefile /home/ckung/Documents/Code/NDSL/examples/serialbox/Fortran/fillq2zero.F90

program testSerialBox

  implicit none

  real, dimension(:,:,:), allocatable :: Qin_out, MASS
  real, dimension(:,:),   allocatable :: FILLQ_out

  integer :: N = 5

  allocate(Qin_out(N,N,N), MASS(N,N,N), FILLQ_out(N,N))

  call random_number(Qin_out)
  call random_number(MASS)

  where(Qin_out < 0.1) Qin_out = -Qin_out

  print*, 'sum(Qin_out) = ', sum(Qin_out)
  print*, 'sum(MASS) = ', sum(MASS)


!$ser init directory='.' prefix='FILLQ2ZERO_In'
!$ser savepoint sp1 it=7
!$ser mode write
!$ser data q=Qin_out m=MASS fq=FILLQ_out
!$ser cleanup

  call FILLQ2ZERO1(Qin_out, MASS, FILLQ_out)

!$ser init directory='.' prefix='FILLQ2ZERO_Out'
!$ser savepoint sp2 it=8
!$ser data q_out=Qin_out m_out=MASS fq_out=FILLQ_out

  print*, 'sum(Qin_out) = ', sum(Qin_out)
  print*, 'sum(FILLQ_out) = ', sum(FILLQ_out)

   contains

  subroutine FILLQ2ZERO1( Q, MASS, FILLQ  )
    real, dimension(:,:,:),   intent(inout)  :: Q
    real, dimension(:,:,:),   intent(in)     :: MASS
    real, dimension(:,:),     intent(  out)  :: FILLQ
    integer                                  :: IM,JM,LM
    integer                                  :: I,J,K,L
    real                                     :: TPW, NEGTPW
    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    ! Fills in negative q values in a mass conserving way.
    ! Conservation of TPW was checked.
    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    IM = SIZE( Q, 1 )
    JM = SIZE( Q, 2 )
    LM = SIZE( Q, 3 )
    do j=1,JM
       do i=1,IM
          TPW = SUM( Q(i,j,:)*MASS(i,j,:) )
          NEGTPW = 0.
          do l=1,LM
             if ( Q(i,j,l) < 0.0 ) then
                NEGTPW   = NEGTPW + ( Q(i,j,l)*MASS( i,j,l ) )
                Q(i,j,l) = 0.0
             endif
          enddo
          do l=1,LM
             if ( Q(i,j,l) >= 0.0 ) then
                Q(i,j,l) = Q(i,j,l)*( 1.0+NEGTPW/(TPW-NEGTPW) )
             endif
          enddo
          FILLQ(i,j) = -NEGTPW
       end do
    end do
  end subroutine FILLQ2ZERO1
end program

The next step is to run `pp_ser.py`, which will replace the `!ser` directive statements will the appropriate Fortran Serialbox calls and write a new `fillq2zero.F90` file into the directory `sb`.

In [ ]:
%%bash

cd /home/ckung/Documents/Code/NDSL/examples/serialbox/Fortran
if [ ! -d "./sb" ]; then
    mkdir sb
fi

python /home/ckung/Documents/Code/SMT-Nebulae/sw_stack/discover/sles15/src/2024.03.00/install/serialbox/python/pp_ser/pp_ser.py --output-dir=./sb fillq2zero.F90

We can go in the `sb` directory and find a `fillq2zero.F90` file that contains the appropriate Serialbox calls.  Building the code with Serialbox enabled requires the following during compilation:

- Referencing to the following libraries
    - `libSerialboxFortran.a`
    - `libSerialboxC.a`
    - `libSerialboxCore.a`
    - `-lstdc++`
- The `-DSERIALIZE` macro
- Adding the Serialbox `include` path

The compilation line looks as follows.

In [ ]:
%%bash

cd /home/ckung/Documents/Code/NDSL/examples/serialbox/Fortran/sb
gfortran fillq2zero.F90  \
    /home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/lib/libSerialboxFortran.a \
    /home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/lib/libSerialboxC.a \
    /home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/lib/libSerialboxCore.a \
    -lstdc++ \
    -DSERIALIZE \
    -I/home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/include \
    -o FILLQ2ZERO.bin


Execute Code.

In [ ]:
%%bash

cd /home/ckung/Documents/Code/NDSL/examples/serialbox/Fortran/sb
./FILLQ2ZERO.bin